In [25]:
import pandas as pd

school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

school_data_df = pd.read_csv(school_data_to_load)
students_data_df = pd.read_csv(student_data_to_load)

student_names = students_data_df["student_name"].tolist()

students_to_fix = []

students_fixed = []


# Add each prefix and suffix to remove to a list.
prefixes_suffixes = ["Dr. ", "Mr. ","Ms. ", "Mrs. ", "Miss ", " MD", " DDS", " DVM", " PhD"]

# Iterate through the words in the "prefixes_suffixes" list and replace them with an empty space, "".
for word in prefixes_suffixes:
    students_data_df["student_name"] = students_data_df["student_name"].str.replace(word,"")


# Put the cleaned students' names in another list.
student_names = students_data_df["student_name"].tolist()


# Create a new list and use it for the for loop to iterate through the list.


# Use an if statement to check the length of the name.

# If the name is greater than or equal to 3, add the name to the list.

for name in student_names:
    if len(name.split())>=3:
        students_fixed.append(name)

len(students_fixed)


school_data_complete_df = pd.merge(students_data_df, school_data_df, on="school_name")

# Get the total number of students.
school_count = len(school_data_complete_df["school_name"].unique())
student_count = school_data_complete_df["Student ID"].count()



total_budget = school_data_df["budget"].sum()

average_reading_score = school_data_complete_df["reading_score"].mean()

average_math_score = school_data_complete_df["math_score"].mean()

passing_math = school_data_complete_df[school_data_complete_df["math_score"] >= 70]

passing_reading = school_data_complete_df[school_data_complete_df["reading_score"] >= 70]


passing_math_count = passing_math["student_name"].count()

passing_reading_count = passing_reading["student_name"].count()

passing_math_percentage = passing_math_count / float(student_count)*100

passing_reading_percentage = passing_reading_count / float(student_count)*100



passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70) & (school_data_complete_df["reading_score"] >= 70)]

overall_passing_math_reading_count = passing_math_reading["student_name"].count()

overall_passing_percentage = overall_passing_math_reading_count/student_count*100


#Creating summary output
district_summary_df = pd.DataFrame(
                [{"Total Schools":school_count,
                 "Total Students":student_count,
                 "Total Budget": total_budget,
                 "Average Math Score": average_math_score,
                 "Average Reading Score": average_reading_score,
                 "% Passing Math": passing_math_percentage,
                 "% Passing Reading": passing_reading_percentage,
                 "% Overall Passing": overall_passing_percentage}])

#Formatting outputs


district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)

district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].map("{:.1f}".format)

district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].map("{:.1f}".format)

district_summary_df["% Passing Math"] = district_summary_df["% Passing Math"].map("{:.0f}".format)

district_summary_df["% Passing Reading"] = district_summary_df["% Passing Reading"].map("{:.0f}".format)

district_summary_df["% Overall Passing"] = district_summary_df["% Overall Passing"].map("{:.0f}".format)


#setting order of columns
new_column_order = [
    "Total Schools",
    "Total Students",
    "Total Budget",
    "Average Math Score",
    "Average Reading Score",
    "% Passing Math",
    "% Passing Reading",
    "% Overall Passing",]

district_summary_df = district_summary_df[new_column_order]

per_school_types = school_data_df.set_index("school_name")["type"]

df = pd.DataFrame(per_school_types)


#This does the
per_school_counts = school_data_df.set_index(["school_name"])["size"]


#same as this
per_school_counts2 = school_data_complete_df["school_name"].value_counts()



#Find the Budget per Student per school

per_school_budget = school_data_df.set_index(["school_name"])["budget"]

per_school_capita = per_school_budget / per_school_counts



#Find average rading and math scores per school
# Calculate the math scores.
student_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]


per_school_reading = school_data_complete_df.groupby(["school_name"]).mean()["reading_score"]


per_school_math = school_data_complete_df.groupby(["school_name"]).mean()["math_score"]


#Calculate passing scores by creating filtered DataFrame

per_school_passing_math = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)]

per_school_passing_reading = school_data_complete_df[(school_data_complete_df["reading_score"] >= 70)]


# Calculate the number of students passing math and passing reading by school.
per_school_passing_math = per_school_passing_math.groupby(["school_name"]).count()["student_name"]

per_school_passing_reading = per_school_passing_reading.groupby(["school_name"]).count()["student_name"]


# Calculate the percentage of passing math and reading scores per school.
per_school_passing_math = per_school_passing_math / per_school_counts *100

per_school_passing_reading = per_school_passing_reading / per_school_counts *100

per_school_passing_math_reading = school_data_complete_df[(school_data_complete_df["math_score"] >= 70)&(school_data_complete_df["reading_score"] >= 70)]

per_school_passing_math_reading = per_school_passing_math_reading.groupby(["school_name"]).count()["student_name"]

per_school_passing_math_reading= per_school_passing_math_reading / per_school_counts*100


per_school_summary_df = pd.DataFrame(
                {"School Types":per_school_types,
                 "Total Students":student_count,
                 "Total School Budget": per_school_budget,
                 "Per Student Budget": per_school_capita,
                 "Average Math Score": per_school_math,
                 "Average Reading Score": per_school_reading,
                 "% Passing Math": per_school_passing_math,
                 "% Passing Reading": per_school_passing_reading,
                 "% Overall Passing": per_school_passing_math_reading})

# Reorder the columns
new_column_order = ["School Types", "Total Students", "Total School Budget", "Per Student Budget", 
                    "Average Math Score", "Average Reading Score", "% Passing Math", 
                    "% Passing Reading","% Overall Passing"]

# Assign district summary df the new column order.
per_school_summary = per_school_summary_df[new_column_order]

#Formatting per_school_summary
per_school_summary_df["Total School Budget"] = per_school_summary_df["Total School Budget"].map("${:,.2f}".format)





top_schools = per_school_summary_df.sort_values(["% Overall Passing"], ascending=False)

bottom_schools = per_school_summary_df.sort_values(["% Overall Passing"])



#Breakdown by grade

ninth_graders = school_data_complete_df[(school_data_complete_df)["grade"]=="9th"]
tenth_graders = school_data_complete_df[(school_data_complete_df)["grade"]=="10th"]
eleventh_graders = school_data_complete_df[(school_data_complete_df)["grade"]=="11th"]
twelfth_graders = school_data_complete_df[(school_data_complete_df)["grade"]=="12th"]




#Breakdown of average MATH by grade
ninth_grade_math_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_grade_math_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

## Adding to dataframe
math_scores_by_grade = pd.DataFrame(
                {"9th":ninth_grade_math_scores,
                 "10th":tenth_grade_math_scores,
                 "11th": eleventh_grade_math_scores,
                 "12th": twelfth_grade_math_scores,})
### Formatting
math_scores_by_grade["9th"] = math_scores_by_grade["9th"].map("{:.1f}".format)
math_scores_by_grade["10th"] = math_scores_by_grade["10th"].map("{:.1f}".format) 
math_scores_by_grade["11th"] = math_scores_by_grade["11th"].map("{:.1f}".format) 
math_scores_by_grade["12th"] = math_scores_by_grade["12th"].map("{:.1f}".format) 

#### Column Order
math_scores_by_grade = math_scores_by_grade[["9th","10th","11th","12th"]]

##### Remove 1st column header
math_scores_by_grade.index.name = None



# Breakdown of average READING by grade
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_grade_reading_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

## Adding to dataframe
reading_scores_by_grade = pd.DataFrame(
                {"9th":ninth_grade_reading_scores,
                 "10th":tenth_grade_reading_scores,
                 "11th": eleventh_grade_reading_scores,
                 "12th": twelfth_grade_reading_scores,})
### Formatting


#### Column Order
reading_scores_by_grade = reading_scores_by_grade[["9th","10th","11th","12th"]]

##### Remove 1st column header
reading_scores_by_grade.index.name = None

##################################################################################################


# Establish the spending bins and group names.
spending_bins = [0,585,630,645,675]

group_names = ["<$586", "$586-630", "$631-645", "$646-675"]

## Cut the per_school_capita into the spending ranges.
per_school_capita.groupby(pd.cut(per_school_capita, spending_bins)).count()

### Categorize spending based on the bins.
per_school_summary_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=group_names)

list(per_school_summary_df.columns)

spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]


spending_math_scores


# Calculate averages for the desired columns.
spending_math_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]

spending_reading_scores = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]

spending_passing_math = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]

spending_passing_reading = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]

overall_passing_spending = per_school_summary_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

overall_passing_spending



spending_summary_df = pd.DataFrame({
                    "Average Math Score": spending_math_scores,
                    "Average Reading Score": spending_math_scores,
                    "% Passing Math": spending_math_scores,
                    "% Passing Reading": spending_math_scores,
                    "% Overall Passing": spending_math_scores})


spending_summary_df["Average Math Score"] = spending_summary_df["Average Math Score"].map("{:.1f}".format)
spending_summary_df["Average Reading Score"] = spending_summary_df["Average Reading Score"].map("{:.1f}".format)
spending_summary_df["% Passing Math"] = spending_summary_df["% Passing Math"].map("{:.0f}".format)
spending_summary_df["% Passing Reading"] = spending_summary_df["% Passing Reading"].map("{:.0f}".format)
spending_summary_df["% Overall Passing"] = spending_summary_df["% Overall Passing"].map("{:.0f}".format)

spending_summary_df

#############################################################################################

per_school_counts

per_school_summary_df

# Establish the spending bins and group names.
size_bins = [0, 999, 1999, 5000]
group_names = ["Small (<1000)", "Medium (1000-1999)", "Large (2000-5000)"]


per_school_counts.groupby(pd.cut(per_school_counts, size_bins)).count()


# Categorize spending based on the bins.
per_school_summary_df["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels=group_names)

per_school_summary_df





C:\Users\Chapm\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: The default value of regex will change from True to False in a future version.


,School Types,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student),School Size
school_name,,,,,,,,,,,
Bailey High School,District,39170,"$3,124,928.00",628.0,77.048432,81.033963,66.680064,81.933280,54.642283,$586-630,NaN
Cabrera High School,Charter,39170,"$1,081,356.00",582.0,83.061895,83.975780,94.133477,97.039828,91.334769,<$586,NaN
Figueroa High School,District,39170,"$1,884,411.00",639.0,76.711767,81.158020,65.988471,80.739234,53.204476,$631-645,NaN
Ford High School,District,39170,"$1,763,916.00",644.0,77.102592,80.746258,68.309602,79.299014,54.289887,$631-645,NaN
Griffin High School,Charter,39170,"$917,500.00",625.0,83.351499,83.816757,93.392371,97.138965,90.599455,$586-630,NaN
Hernandez High School,District,39170,"$3,022,020.00",652.0,77.289752,80.934412,66.752967,80.862999,53.527508,$646-675,NaN
Holden High School,Charter,39170,"$248,087.00",581.0,83.803279,83.814988,92.505855,96.252927,89.227166,<$586,NaN
Huang High School,District,39170,"$1,910,635.00",655.0,76.629414,81.182722,65.683922,81.316421,53.513884,$646-675,NaN
Johnson High School,District,39170,"$3,094,650.00",650.0,77.072464,80.966394,66.057551,81.222432,53.539172,$646-675,NaN
